# Matriz Origen Destino

En el notebook anterior calculamos las trayectorias de los usuarios de Twitter. En este notebook vamos a obtener la matriz Origen-Destino a partir de dichas trayectorias.

In [1]:
%load_ext autoreload
%autoreload 2
import geopandas as gpd
from shapely.geometry import LineString, Point
from datetime import datetime, timedelta
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from IPython.display import display, HTML
import contextily as ctx

Leemos las trayectorias que calculamos

In [3]:
trayectorias = gpd.read_file("../output/trayectorias_2018.gpkg")
trayectorias['dia'] = pd.to_datetime(trayectorias['dia'], format='%Y-%m-%dT%H:%M:%S')
trayectorias['intervalo'] = pd.to_datetime(trayectorias['intervalo'], format='%Y-%m-%dT%H:%M:%S')
trayectorias.set_index(['dia', 'intervalo', 'Usuario'], inplace=True)
trayectorias.head()

separation  \
dia                       intervalo                 Usuario                     
2018-01-01 00:00:00+00:00 2017-12-31 22:00:00+00:00 2Rene1982     4776.501901   
                                                    AdryLATREVI   9317.335796   
                                                    BoogeymanMX   4253.572117   
                                                    Calcifer_May  9841.699738   
                                                    ChalArauz     2779.291581   

                                                                                                           geometry  
dia                       intervalo                 Usuario                                                          
2018-01-01 00:00:00+00:00 2017-12-31 22:00:00+00:00 2Rene1982     LINESTRING (486006.893 2148777.685, 490781.385...  
                                                    AdryLATREVI   LINESTRING (482869.514 2147672.967, 486107.871...  
                                                    BoogeymanMX   LINESTRING (490531.670 2170098.393, 486303.994...  
                                                    Calcifer_May  LINESTRING (486006.893 2148777.685, 486107.871...  
                                                    ChalArauz     LINESTRING (487840.819 2158221.322, 486714.205...

Y las zonas de análisis

In [4]:
zonas = gpd.read_file("../data/shapes/DistritosEODHogaresZMVM2017.shp")
zonas.head()

,Distrito,Descripcio,geometry
0,001,Centro HistÃ³rico,"POLYGON ((485707.699 2149605.446, 485792.518 2..."
1,002,Buenavista-Reforma,"POLYGON ((485002.129 2152136.266, 485069.252 2..."
2,003,Tlatelolco,"POLYGON ((486206.963 2152150.606, 486612.741 2..."
3,004,Morelos,"POLYGON ((488510.758 2150946.712, 488607.167 2..."
4,005,"Moctezuma, Terminal de Autobuses de Oriente (T...","POLYGON ((489445.177 2150604.177, 489535.192 2..."


## Un dia y un intervalo

Primero vamos a calcular la matriz para un sólo dia y un intervalo. 

In [6]:
prueba = trayectorias.loc['2018-05-24 00:00:00','2018-05-24 06:00:00+00:00',:].copy()
prueba.head()

separation  \
dia                       intervalo                 Usuario                        
2018-05-24 00:00:00+00:00 2018-05-24 06:00:00+00:00 CavaMildred      5858.563304   
                                                    Danny_Figueroa   6112.225570   
                                                    ErnestoRUrbina  10487.216134   
                                                    JonhyLoConoci   12547.768991   
                                                    NicioLaura      15782.687224   

                                                                                                             geometry  
dia                       intervalo                 Usuario                                                            
2018-05-24 00:00:00+00:00 2018-05-24 06:00:00+00:00 CavaMildred     LINESTRING (483997.931 2137169.995, 483043.193...  
                                                    Danny_Figueroa  LINESTRING (477796.301 2148413.500, 483900.425...  
                                                    ErnestoRUrbina  LINESTRING (483970.238 2136319.381, 483970.238...  
                                                    JonhyLoConoci   LINESTRING (477796.301 2148413.500, 490781.385...  
                                                    NicioLaura      LINESTRING (477796.301 2148413.500, 467899.291...

Para cada usuario tenemos que encontrar los puntos de origen y destino de las trayectorias, para eso podemos crear dos dataframes diferentes, una para los orígenes y otra para los destinos.

In [8]:
origen = prueba['geometry'].apply(lambda l: Point(l.coords[0])).reset_index()
origen = origen.set_crs(epsg=32614)
destino = prueba['geometry'].apply(lambda l: Point(l.coords[-1])).reset_index()
destino = destino.set_crs(epsg=32614)

Y asignamos las zonas a cada uno de ellos 

In [9]:
origen = gpd.sjoin(origen.reset_index(), zonas).drop(['index', 'index_right', 'Descripcio'], axis=1)
destino = gpd.sjoin(destino.reset_index(), zonas).drop(['index','index_right', 'Descripcio'], axis=1)
origen.head()

,dia,intervalo,Usuario,geometry,Distrito
0,2018-05-24 00:00:00+00:00,2018-05-24 06:00:00+00:00,CavaMildred,POINT (483997.931 2137169.995),049
2,2018-05-24 00:00:00+00:00,2018-05-24 06:00:00+00:00,ErnestoRUrbina,POINT (483970.238 2136319.381),049
7,2018-05-24 00:00:00+00:00,2018-05-24 06:00:00+00:00,Piccolodmk,POINT (483997.931 2137169.995),049
12,2018-05-24 00:00:00+00:00,2018-05-24 06:00:00+00:00,labelpacklat,POINT (483970.238 2136319.381),049
1,2018-05-24 00:00:00+00:00,2018-05-24 06:00:00+00:00,Danny_Figueroa,POINT (477796.301 2148413.500),058


Realmente sólo necesitamos conservar los índices temporales (día e intervalo), el `Usuario` y los distritos de origen y destino. Cada viaje está identificado por el `Usuario`

In [10]:
origen_destino = origen[['dia', 'intervalo', 'Usuario' , 'Distrito']].merge(destino[['Usuario' , 'Distrito']], on='Usuario')
origen_destino.rename({'Distrito_x':'distrito_origen', 'Distrito_y':'distrito_destino'}, axis=1, inplace=True)
origen_destino.head()

,dia,intervalo,Usuario,distrito_origen,distrito_destino
0,2018-05-24 00:00:00+00:00,2018-05-24 06:00:00+00:00,CavaMildred,049,015
1,2018-05-24 00:00:00+00:00,2018-05-24 06:00:00+00:00,ErnestoRUrbina,049,037
2,2018-05-24 00:00:00+00:00,2018-05-24 06:00:00+00:00,Piccolodmk,049,115
3,2018-05-24 00:00:00+00:00,2018-05-24 06:00:00+00:00,labelpacklat,049,062
4,2018-05-24 00:00:00+00:00,2018-05-24 06:00:00+00:00,Danny_Figueroa,058,002


Ya sólo necesitamos agregar por origen y destino y contar la cantidad de viajes en cada grupo

In [11]:
origen_destino = origen_destino.groupby(['distrito_origen', 'distrito_destino']).count()
origen_destino.head(20)

dia  intervalo  Usuario
distrito_origen distrito_destino                         
001             123                 1          1        1
002             015                 1          1        1
049             015                 1          1        1
                037                 1          1        1
                062                 1          1        1
                115                 1          1        1
058             002                 1          1        1
                009                 1          1        1
                037                 1          1        1
                060                 1          1        1
067             002                 1          1        1
                062                 1          1        1
121             111                 1          1        1

Esto que hicimos fua sólo para un dia y un intervalo, ahora hay que hacerlo para todos. En el `DataFrame` resulante tenemos que tener, para cada dia e intervalo, un resultado igual al que tenemos arriba en `origen_destino`

## Para todos los dias

Lo único que necesitamos hacer para obtener el resultado que buuscamos es definir una fución de agregación que regrese, para cada dia e intervalo, un `DataFrame` como el que tenemos arriba

In [14]:
def get_od(df, zonas):    
    origen = (df['geometry']
             .apply(lambda l: Point(l.coords[0]))
             .reset_index()
             .set_crs(epsg=32614)
             )
    destino = (df['geometry']
              .apply(lambda l: Point(l.coords[-1]))
              .reset_index()
              .set_crs(epsg=32614)
              )
    origen = (gpd.sjoin(origen.reset_index(), zonas)
             .drop(['index', 'index_right', 'Descripcio'], axis=1)
             )
    destino = (gpd.sjoin(destino.reset_index(), zonas)
              .drop(['index','index_right', 'Descripcio'], axis=1)
              )
    origen_destino = (origen[['dia', 'intervalo', 'Usuario' , 'Distrito']]
                     .merge(destino[['Usuario' , 'Distrito']], on='Usuario')
                     .rename({'Distrito_x':'distrito_origen', 'Distrito_y':'distrito_destino'}
                             ,axis=1)
                     )
    origen_destino = origen_destino.groupby(['distrito_origen', 'distrito_destino']).count()
    return origen_destino



In [15]:
od_final = trayectorias.groupby(level=['dia', 'intervalo']).apply(get_od, zonas=zonas)
od_final.head()

dia  \
dia                       intervalo                 distrito_origen distrito_destino        
2018-01-01 00:00:00+00:00 2017-12-31 22:00:00+00:00 001             001                 1   
                                                                    002                 1   
                                                                    034                 1   
                                                                    044                 1   
                                                                    046                 1   

                                                                                      intervalo  \
dia                       intervalo                 distrito_origen distrito_destino              
2018-01-01 00:00:00+00:00 2017-12-31 22:00:00+00:00 001             001                       1   
                                                                    002                       1   
                                                                    034                       1   
                                                                    044                       1   
                                                                    046                       1   

                                                                                      Usuario  
dia                       intervalo                 distrito_origen distrito_destino           
2018-01-01 00:00:00+00:00 2017-12-31 22:00:00+00:00 001             001                     1  
                                                                    002                     1  
                                                                    034                     1  
                                                                    044                     1  
                                                                    046                     1

Arreglemos los nombres de las columnas, en realidad todas son iguales y representan el número de viajes.

In [16]:
od_final = (od_final[['dia']]
           .rename({'dia':'viajes'}, axis=1)
           )
od_final.head()

viajes
dia                       intervalo                 distrito_origen distrito_destino        
2018-01-01 00:00:00+00:00 2017-12-31 22:00:00+00:00 001             001                    1
                                                                    002                    1
                                                                    034                    1
                                                                    044                    1
                                                                    046                    1

## Agregados por semana

Para cada dia las muestras son relativamente pequeñas, entonces vamos a sacar primero los agregados para cada semana y luego obtener el promedio final. Recordemos que aquí tenemos todos los dias del periodo muestreado y que, en principio vamos a tener dos tipos de comportamiento diferentes: dias laborales y fines de semana. Entonces, primero, vamos a obtener la matriz sólo para los días laborales. Antes de eso tenemos que ajustar el índice de intervalo porque incluye el dia, entonces no podemos agrupar bien

In [17]:
od_final = od_final.reset_index()
od_final['intervalo_time'] = od_final['intervalo'].apply(lambda x: x.strftime('%H:%M:%S'))
od_final = (od_final.drop(['intervalo'], axis=1)
            .rename({'intervalo_time':'intervalo'}, axis=1)
            .set_index(['dia', 'intervalo', 'distrito_origen'])
           )
od_final

distrito_destino  viajes
dia                       intervalo distrito_origen                         
2018-01-01 00:00:00+00:00 22:00:00  001                          001       1
                                    001                          002       1
                                    001                          034       1
                                    001                          044       1
                                    001                          046       1
...                                                              ...     ...
2018-12-31 00:00:00+00:00 22:00:00  111                          115       1
                                    131                          137       1
                                    139                          034       1
                                    139                          173       1
                                    158                          147       1

[151727 rows x 2 columns]

Seleccionemos los dias laborales

In [18]:
od_laborales = od_final.loc[od_final.index.get_level_values('dia').weekday.isin(range(0,5)),:,:]

Agrupemos estos datos por semana 

In [19]:
laborales_semana = od_final.groupby([pd.Grouper(level='dia', freq='W'), 'intervalo', 'distrito_origen', 'distrito_destino']).sum()
laborales_semana

viajes
dia                       intervalo distrito_origen distrito_destino        
2018-01-07 00:00:00+00:00 02:00:00  001             001                   12
                                                    002                   11
                                                    003                    2
                                                    006                    1
                                                    008                    4
...                                                                      ...
2019-01-06 00:00:00+00:00 22:00:00  139             173                    1
                                    158             147                    1
                                    171             171                    1
                                    173             045                    1
                                    207             206                    1

[84588 rows x 1 columns]

Finalmente, calculemos los estadísticos (suma, media, desviación estándar) para cada uno de los intervalos sobre todo el periodo de estudio.

In [20]:
promedios_intervalo = laborales_semana.groupby(['intervalo', 'distrito_origen', 'distrito_destino']).agg(['sum', 'mean', 'std'])
promedios_intervalo

viajes                     
                                              sum       mean       std
intervalo distrito_origen distrito_destino                            
02:00:00  001             001                 410   7.735849  4.015120
                          002                 238  17.000000  6.598368
                          003                  45   1.500000  0.682288
                          004                  11   1.100000  0.316228
                          005                  14   1.000000  0.000000
...                                           ...        ...       ...
22:00:00  300             134                   1   1.000000       NaN
                          135                   1   1.000000       NaN
                          165                   4   1.333333  0.577350
                          171                   1   1.000000       NaN
                          300                   6   1.000000  0.000000

[16758 rows x 3 columns]

Exportamos el resultado como csv

In [21]:
promedios_intervalo.reset_index(col_level=1).to_csv("../output/matriz_od_semana_2018.csv")